** In this Jupyter Notebook, the absolute + sensitivity analysis results are generated for a One-at-a-time sensitivity analysis of transition year for the proliferation risk evaluation metric. The reason why this metric is separated from the others is that you need the -exp version of each sqlite and pyne on your machine to make it work **

In [1]:
import cymetric as cym
from cymetric import timeseries
import matplotlib as plt 
import pandas as pd
import numpy as np
import sys 
sys.path.insert(0, '../../../scripts/')
import output as oup

/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


# This is when you already have the results 

In [5]:
df_p = pd.read_csv('ty-df-pu.csv',index_col='TY')
df_p

,Max Pu in all CP,Pu Quality in all CP at Max Pu,Max Pu in HLW,Pu Quality in HLW at Max Pu,Max Pu in all RPR,Pu Quality in all RPR at Max Pu
TY,,,,,,
960,285794.85,0.59,42905.59,0.62,2552142.82,0.48
972,269666.80,0.60,44291.11,0.62,2530146.38,0.48
984,273313.41,0.60,45673.72,0.62,2489776.12,0.48
996,273307.72,0.59,46848.37,0.62,2475855.63,0.48
1008,269414.41,0.60,46714.21,0.62,2455878.54,0.48


# The rest of the code below is to generate the above results

In [3]:
starter_string = 'TY'
scenario_nums = ['960','972','984','996','1008']

In [4]:
df_p = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [ ]:
df_p['Final HLW'] = 0
df_p['Final Depleted U'] = 0 
df_p['Total uranium ore'] = 0
df_p['Total idle capacity'] = 0
df_p['Last date idle capacity'] = 0
df_p['Duration of transition'] = 0

In [ ]:
output_start = '../cyclus-files/oat/transition-year/ty'
ev_dict = {}
for x in range(len(scenario_nums)): 
    output_file = output_start + scenario_nums[x]+'-exp.sqlite'
    ev_dict[scenario_nums[x]] = cym.Evaluator(db=cym.dbopen(output_file),write=True)

In [ ]:
for x in range(len(scenario_nums)): 
    cp = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage','moxstorage','frstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity']
    fissile_cp = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage','moxstorage','frstorage'],nucs=['pu-239','pu-241'])['Quantity']
    df_p.loc[scenario_nums[x],'Max Pu in all CP'] = cp.max()
    df_p.loc[scenario_nums[x],'Pu Quality in all CP at Max Pu'] = fissile_cp[cp.idxmax()]/cp.max()
    hlw = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['enrichmentsink','lwrsink','moxsink','frsink'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity']
    fissile_hlw = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['enrichmentsink','lwrsink','moxsink','frsink'],nucs=['pu-239','pu-241'])['Quantity']
    df_p.loc[scenario_nums[x],'Max Pu in HLW'] = hlw.max()
    df_p.loc[scenario_nums[x],'Pu Quality in HLW at Max Pu'] = fissile_hlw[hlw.idxmax()]/hlw.max()
    rpr = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrreprocessing','moxreprocessing','frreprocessing'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity']
    fissile_rpr = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrreprocessing','moxreprocessing','frreprocessing'],nucs=['pu-239','pu-241'])['Quantity']
    df_p.loc[scenario_nums[x],'Max Pu in all RPR'] = rpr.max()
    df_p.loc[scenario_nums[x],'Pu Quality in all RPR at Max Pu'] = fissile_rpr[rpr.idxmax()]/rpr.max()

In [ ]:
df_p.to_csv('ty-df-pu.csv')

In [6]:
df_p_sa = oup.sensitivity(960,df_p)
df_p_sa

,Max Pu in all CP,Pu Quality in all CP at Max Pu,Max Pu in HLW,Pu Quality in HLW at Max Pu,Max Pu in all RPR,Pu Quality in all RPR at Max Pu
TY,,,,,,
960,0.000000,0.000000,0.000000,0.0,0.000000,0.0
972,-5.643226,1.694915,3.229230,0.0,-0.861881,0.0
984,-4.367273,1.694915,6.451677,0.0,-2.443699,0.0
996,-4.369263,0.000000,9.189432,0.0,-2.989143,0.0
1008,-5.731538,1.694915,8.876745,0.0,-3.771900,0.0


In [7]:
df_p_sa.to_csv('ty-df-pu-sa.csv')